In [1]:
import sympy as sp
import numpy as np
import pandas as pd

In [55]:
x = sp.symbols('x')
f = sp.exp(2 * x)

In [43]:
def calculate_first_derivative(nodes, values, forward=True):
    step = nodes[1] - nodes[0]
    diffs = []
    if forward:
        for i in range(len(nodes) - 1):
            diffs.append((values[i + 1] - values[i]) / step)
        diffs.append(None)
    else:
        diffs.append(None)
        for i in range(1, len(nodes)):
            diffs.append((values[i] - values[i - 1]) / step)
    
    return diffs

In [44]:
def calculate_first_derivative_high_precision(nodes, values):
    step = nodes[1] - nodes[0]
    diffs = [None]
    for i in range(1, len(nodes) - 1):
        diffs.append((values[i + 1] - values[i - 1]) / (2 * step))
    diffs.append(None)
    return diffs

In [45]:
def calculate_second_derivative(nodes, values):
    step = nodes[1] - nodes[0]
    diffs = [None]
    for i in range(1, len(nodes) - 1):
        diffs.append((values[i - 1] - 2 * values[i] + values[i + 1]) / step ** 2)
    diffs.append(None)
    return diffs

In [53]:
def create_table_of_derivative_estimation(nodes, values, dvalues, d_est, d_est_hp, ddvalues, dd_est):
    derror = [abs(v - e) if e != None else None for v, e in zip(dvalues, d_est)]
    dhperror = [abs(v - e) if e != None else None for v, e in zip(dvalues, d_est_hp)]
    dderror = [abs(v - e) if e != None else None for v, e in zip(ddvalues, dd_est)]
    
    rows = list(zip(
        nodes, values,
        dvalues,
        d_est, derror,
        d_est_hp, dhperror,
        ddvalues,
        dd_est, dderror
    ))
    header = ['x', 'f(x)',
              'f\'(x)',
              'f\'(x) O(n) est.', 'f\'(x) O(n) error',
              'f\'(x) O(n^2) est.', 'f\'(x) O(n^2) error',
              'f\'\'(x)',
              'f\'\'(x) O(n^2) est.', 'f\'(x) O(n^2) error'
             ]
    return pd.DataFrame(rows, columns=header)

# 1

In [58]:
a = 0
b = 4
nodes_count = 11
df = sp.diff(f)
ddf = sp.diff(df)

nodes = np.linspace(a, b, nodes_count, endpoint=True)
values = [f.evalf(subs={x: i}) for i in nodes]
dvalues = [df.evalf(subs={x: i}) for i in nodes]
ddvalues = [ddf.evalf(subs={x: i}) for i in nodes]

d_est = calculate_first_derivative(nodes, values)
d_est_hp = calculate_first_derivative_high_precision(nodes, values)
dd_est = calculate_second_derivative(nodes, values)

In [59]:
create_table_of_derivative_estimation(nodes, values, dvalues, d_est, d_est_hp, ddvalues, dd_est)

,x,f(x),f'(x),f'(x) O(n) est.,f'(x) O(n) error,f'(x) O(n^2) est.,f'(x) O(n^2) error,f''(x),f''(x) O(n^2) est.,f'(x) O(n^2) error
0,0.0,1.00000000000000,2.00000000000000,3.06385232123117,1.06385232123117,None,None,4.00000000000000,None,None
1,0.4,2.22554092849247,4.45108185698494,6.81872873975662,2.36764688277168,4.94129053049389,0.490208673508958,8.90216371396987,9.38719104631362,0.485027332343746
2,0.8,4.95303242439511,9.90606484879023,15.1753598906162,5.26929504182599,10.9970443151864,1.09097946639619,19.8121296975805,20.8915778771490,1.07944817956855
3,1.2,11.0231763806416,22.0463527612832,33.7733845411694,11.7270317798862,24.4743722158928,2.42801945460959,44.0927055225664,46.4950616263828,2.40235610381643
4,1.6,24.5325301971094,49.0650603942187,75.1640495900872,26.0989891958685,54.4687170656283,5.40365667140959,98.1301207884374,103.476662622295,5.34654183385715
5,2.0,54.5981500331442,109.196300066288,167.280668713977,58.0843686476883,121.222359152032,12.0260590857435,218.392600132577,230.291547809724,11.8989476771470
6,2.4,121.510417518735,243.020835037470,372.289974768544,129.269139731074,269.785321741261,26.7644867037907,486.041670074940,512.523265136419,26.4815950614791
7,2.8,270.426407426153,540.852814852306,828.546576114824,287.693761262518,600.418275441684,59.5654605893785,1081.70562970461,1140.64150336570,58.9358736610868
8,3.2,601.845037872082,1203.69007574416,1843.96431630584,640.274240561675,1336.25544621033,132.565370466167,2407.38015148833,2538.54435047754,131.164198989210
9,3.6,1339.43076439442,2678.86152878884,4103.81805661828,1424.95652782944,2973.89118646206,295.029657673221,5357.72305757767,5649.63435078109,291.911293203418


# 2

In [71]:
step = 1
target_node = 2
target_value = ddf.evalf(subs={x: target_node})
error = None

while True:
    nodes = [target_node - step, target_node, target_node + step]
    values = [f.evalf(subs={x: i}) for i in nodes]
    estimated_value = calculate_second_derivative(nodes, values)[1]
    new_error = round(abs(estimated_value - target_value), 5)
    
    if error == None or new_error <= error:
        error = new_error
        step /= 2
    else:
        break

In [73]:
step

3.0517578125e-05

# 3

In [87]:
def newton_interpolation_equally_spaced_nodes(nodes, values, order):
    def finite_differences(nodes, values, max_order):
        if len(nodes) != len(values):
            raise ValueError("nodes and values lists must have the same length")
        diffs = [values]
        for order in range(max_order):
            current_diffs = []
            for i in range(1, len(nodes) - order):
                current_diffs.append(diffs[-1][i] - diffs[-1][i-1])
            diffs.append(current_diffs)
        return diffs
        
    step = nodes[1] - nodes[0]
    fds = finite_differences(nodes, values, order)

    t = (x - b) / step

    # Compute N_k for k up `order` to estimate error for polynom of order `order - 1`
    ns = [1]
    for i in range(order + 1):
        ns.append(ns[-1] * (t + i) / (i + 1))

    # Calculate all the polynoms with order from 0 to `order`
    polynoms = []
    for i in range(order + 1):
        prev = polynoms[-1] if len(polynoms) > 0 else 0
        polynoms.append(prev + ns[i] * fds[i][-1])

    return polynoms[-1]

In [126]:
a, b = 0, 1
nodes_count = 11 # to include node `b` and have a nice 0.1 step
nodes = np.linspace(a, b, nodes_count, endpoint=True)
values = [f.evalf(subs={x: i}) for i in nodes]
new_node_id = 1

polynom = newton_interpolation_equally_spaced_nodes(nodes, values, 5)

In [127]:
df_numeric = calculate_first_derivative(nodes, values)[new_node_id]
df_poly = sp.diff(polynom).evalf(subs={x: nodes[new_node_id]})
df_actual = sp.diff(f).evalf(subs={x: nodes[new_node_id]})

In [128]:
df_actual, df_poly, df_numeric

(2.44280551632034, 2.64068217834383, 2.70421939481100)

In [130]:
poly_error, numeric_error = df_poly - df_actual, df_numeric - df_actual
poly_error, numeric_error

(0.197876662023491, 0.261413878490665)